In [1]:
from requests import get
import pandas as pd


In [2]:
all_countries=get('https://raw.githubusercontent.com/everypolitician/everypolitician-data/master/countries.json')
j=all_countries.json()
len(j)

233

In [3]:
len(j)

233

In [4]:
aus_hr_url=j[13]['legislatures'][0]['popolo_url']
aus_hr_data=get(aus_hr_url)
aus_hr_df=pd.json_normalize(aus_hr_data.json()['persons'])

In [5]:
aus_hr_df['birth_date']=pd.to_datetime(aus_hr_df['birth_date'])
aus_hr_df.head()

,birth_date,contact_details,family_name,gender,given_name,id,identifiers,links,name,sort_name,death_date,other_names,email,image,images,honorific_prefix
0,1954-03-21,"[{'type': 'twitter', 'value': 'nueday'}]",Kelly,female,De-Anne,004e5075-34b7-4ba5-b53b-f992a28ea6dc,"[{'identifier': '10357', 'scheme': 'everypolit...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",De-Anne Kelly,"Kelly, De-Anne",NaN,NaN,NaN,NaN,NaN,NaN
1,1925-06-15,NaN,Millar,male,Percival,00c2ed39-73db-43aa-a5b7-bea4508dbd5e,"[{'identifier': '10457', 'scheme': 'everypolit...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",Percival Millar,"Millar, Percival",2017-11-28,"[{'lang': 'en', 'name': 'Clarrie Millar', 'not...",NaN,NaN,NaN,NaN
2,1951-07-04,"[{'type': 'email', 'value': 'john.alexander.mp...",Alexander,male,John,010ba615-63c1-466b-a425-5d127e12b8c3,"[{'identifier': 'M3M', 'scheme': 'aph'}, {'ide...","[{'note': 'Wikimedia Commons', 'url': 'https:/...",John Alexander,"Alexander, John",NaN,"[{'lang': 'ar', 'name': 'جون ألكسندر', 'note':...",john.alexander.mp@aph.gov.au,http://www.aph.gov.au//api/parliamentarian/M3M...,[{'url': 'http://www.aph.gov.au//api/parliamen...,NaN
3,1954-02-13,"[{'type': 'twitter', 'value': 'senronno'}]",Ronaldson,male,Michael,012988dd-b93f-4bfe-95b8-27f3e5a56ee7,"[{'identifier': 'xt4', 'scheme': 'aph'}, {'ide...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",Michael Ronaldson,"Ronaldson, Michael",NaN,"[{'lang': 'ja', 'name': 'マイケル・ロナルドソン', 'note':...",NaN,NaN,NaN,NaN
4,1946-08-28,NaN,Andren,male,Peter,0267b78a-440a-414f-bf35-5284b9ebaa0d,"[{'identifier': '10012', 'scheme': 'everypolit...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",Peter Andren,"Andren, Peter",2007-11-03,NaN,NaN,NaN,NaN,NaN


In [6]:
aus_hr_df['birth_date']=pd.to_datetime(aus_hr_df['birth_date'])
monthly_df=aus_hr_df.groupby(aus_hr_df['birth_date'].dt.month).count()[['name']]
monthly_df.index=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
monthly_df['percentage']=(monthly_df['name']/monthly_df['name'].sum()*100).round(1).astype(str) + '%'
monthly_df.rename(columns={'name':'count'},inplace=True)
monthly_df.transpose()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
count,45,38,52,44,40,46,47,41,43,43,38,38
percentage,8.7%,7.4%,10.1%,8.5%,7.8%,8.9%,9.1%,8.0%,8.3%,8.3%,7.4%,7.4%
